# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns
student_data.head(10)

Student data read successfully!


school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4   at_home   teacher   
1     GP   F   17       U     GT3       T     1     1   at_home     other   
2     GP   F   15       U     LE3       T     1     1   at_home     other   
3     GP   F   15       U     GT3       T     4     2    health  services   
4     GP   F   16       U     GT3       T     3     3     other     other   
5     GP   M   16       U     LE3       T     4     3  services     other   
6     GP   M   16       U     LE3       T     2     2     other     other   
7     GP   F   17       U     GT3       A     4     4     other   teacher   
8     GP   M   15       U     LE3       A     3     2  services     other   
9     GP   M   15       U     GT3       T     3     4     other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3        6   
1  ...        yes       no       5         3      3    1    1      3        4   
2  ...        yes       no       4         3      2    2    3      3       10   
3  ...        yes      yes       3         2      2    1    1      5        2   
4  ...         no       no       4         3      2    1    2      5        4   
5  ...        yes       no       5         4      2    1    2      5       10   
6  ...        yes       no       4         4      4    1    1      3        0   
7  ...         no       no       4         1      4    1    1      1        6   
8  ...        yes       no       4         2      2    1    1      1        0   
9  ...        yes       no       5         5      1    1    1      5        0   

  passed  
0     no  
1     no  
2    yes  
3    yes  
4    yes  
5    yes  
6    yes  
7     no  
8    yes  
9    yes  

[10 rows x 31 columns]

Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]
n_passed = np.shape(np.where(student_data.passed == "yes"))[1]                                                    
n_failed = np.shape(np.where(student_data.passed == "no"))[1]
grad_rate = float(n_passed)/float(n_students)*100

print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, train_size = 300, random_state = 2016)

X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(
    X_all, y_all, train_size = 100, random_state = 2016)

X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(
    X_all, y_all, train_size = 200, random_state = 2016)

#X_train = ?
#y_train = ?
#X_test = ?
#y_test = ?
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
# Model 1: Decision Trees
from sklearn import tree
clf_dt = tree.DecisionTreeClassifier(criterion="entropy")

# Model 2: SVM
from sklearn import svm
clf_linSVC = svm.LinearSVC()

# Model 3: Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf_nb = GaussianNB()

# Model 4: Emsemble AbaBoosting
from sklearn.ensemble import AdaBoostClassifier
clf_aba = AdaBoostClassifier(n_estimators=500)

# Fit model to training data
train_classifier(clf_dt, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf_dt, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf_dt, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.727272727273


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

# Decision Tree
train_predict(clf_dt, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_dt, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_dt, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.619469026549
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.717557251908
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 

In [11]:
from sklearn.externals.six import StringIO  
import pydot
dot_data = StringIO() 
tree.export_graphviz(clf_dt, out_file=dot_data, 
                     feature_names=list(X_all.columns), 
                     class_names=['not passed','passed'],
                     filled=True, 
                     rounded=True,
                     special_characters=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("student_intervention_dt.pdf")
graph.write_png("student_intervention_dt.png")

True

In [12]:
from IPython.display import Image
Image(filename='student_intervention_dt.png')

In [13]:
# Naive Bayes
train_predict(clf_nb, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_nb, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_nb, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.654205607477
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.413043478261
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.811594202899
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.724409448819
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.811320754717
Predicting labels using GaussianNB...
Done!
Prediction time

In [14]:
# Support Vector Machine - Linear Support Vector Classification
train_predict(clf_linSVC, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_linSVC, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_linSVC, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 100
Training LinearSVC...
Done!
Training time (secs): 0.008
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.019
F1 score for training set: 0.943661971831
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.707692307692
------------------------------------------
Training set size: 200
Training LinearSVC...
Done!
Training time (secs): 0.015
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.843853820598
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.782608695652
------------------------------------------
Training set size: 300
Training LinearSVC...
Done!
Training time (secs): 0.031
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.849765258216
Predicting labels using LinearSVC...
Done!
Prediction time (secs): 

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [21]:
# TODO: Fine-tune your model and report the best F1 score
# Since Naive Bayes has no parameter, Gridsearch can't be used here
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

clf_1 = clf_nb
clf_2 = MultinomialNB()
clf_3 = BernoulliNB()

all_train_sample = []

for i in range(250):
    all_train_sample.append([X_train[i:],y_train[i:]])

best_size_gnb = []
for each in all_train_sample:
    clf_1.fit(each[0], each[1])
    y_pred = clf_1.predict(X_test)
    best_size_gnb.append(f1_score(y_test.values, y_pred, pos_label='yes'))
    
best_size_mnb = []
for each in all_train_sample:
    clf_2.fit(each[0], each[1])
    y_pred = clf_2.predict(X_test)
    best_size_mnb.append(f1_score(y_test.values, y_pred, pos_label='yes'))   

best_size_bnb = []
for each in all_train_sample:
    clf_3.fit(each[0], each[1])
    y_pred = clf_3.predict(X_test)
    best_size_bnb.append(f1_score(y_test.values, y_pred, pos_label='yes'))   

print "Best test F1 score for Gaussian Naive Bayes is", max(best_size_gnb)
print "Best training size for Gaussian Naive Bayes is", len(all_train_sample[best_size_gnb.index(max(best_size_gnb))][0])
print ""
print "Best test F1 score for Multinomial Naive Bayes is", max(best_size_mnb)
print "Best training size for Multinomial Naive Bayes is", len(all_train_sample[best_size_mnb.index(max(best_size_mnb))][0])
print ""
print "Best test F1 score for Bernoulli Naive Bayes is", max(best_size_bnb)
print "Best training size for Bernoulli Naive Bayes is", len(all_train_sample[best_size_bnb.index(max(best_size_bnb))][0])

Best test F1 score for Gaussian Naive Bayes is 0.765625
Best training size for Gaussian Naive Bayes is 271

Best test F1 score for Multinomial Naive Bayes is 0.780141843972
Best training size for Multinomial Naive Bayes is 115

Best test F1 score for Bernoulli Naive Bayes is 0.791366906475
Best training size for Bernoulli Naive Bayes is 266


In [19]:
#Gridsearch on decision tree
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

clf_df1 = tree.DecisionTreeClassifier(criterion='entropy')

tuned_parameters = {'max_depth': np.arange(3, 15)}

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print ""

    clf = GridSearchCV(clf_df1, tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print ""
    print(clf.best_params_)
    print ""
    print("Grid scores on development set:")
    print ""
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))

# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'max_depth': 3}

Grid scores on development set:

0.706 (+/-0.167) for {'max_depth': 3}
0.670 (+/-0.106) for {'max_depth': 4}
0.684 (+/-0.105) for {'max_depth': 5}
0.704 (+/-0.092) for {'max_depth': 6}
0.680 (+/-0.115) for {'max_depth': 7}
0.684 (+/-0.129) for {'max_depth': 8}
0.673 (+/-0.110) for {'max_depth': 9}
0.663 (+/-0.044) for {'max_depth': 10}
0.668 (+/-0.101) for {'max_depth': 11}
0.671 (+/-0.114) for {'max_depth': 12}
0.660 (+/-0.100) for {'max_depth': 13}
0.653 (+/-0.055) for {'max_depth': 14}
